# Examine variant merge results

In this notebook, we use [Hail](https://hail.is/) to examine the results of the merge of AoU and UKB variants. This notebook is not part of the main analysis and can be skipped entirely. It can also be run interactively on a small partion of the genome to do some basic checking.

Note that this work is part of a larger project to [Demonstrate the Potential for Pooled Analysis of All of Us and UK Biobank Genomic Data](https://github.com/all-of-us/ukb-cross-analysis-demo-project). Specifically this is for the portion of the project that is the **pooled** analysis.

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the <i>All of Us</i> Workbench.
    <ul>
        <li>Use "Recommended Environment" <kbd><b>Hail Genomics Analysis</b></kbd> which creates compute type <kbd>Dataproc Cluster</kbd> with reasonable defaults for CPU, RAM, disk, and number of workers. If you like, you can increase the number of workers to make this job complete faster.</li>
        <li>This notebook is not part of the main analysis and can be skipped entirely.</li>
        <ul>
            <li>It can be run interactively and/or on a small partion of the genome to do some basic checking.</li>
            <li>When run on chr1 - chr22, it took 13 hours with 10 workers with Hail version 0.2.74-0c3a74d12093.</li>
        </ul>
    </ul>
</div>

In [ ]:
from datetime import datetime
import hail as hl
import math
import os
import time

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# Matrix table was provided by AoU.
AOU_MT = 'gs://fc-secure-4adb21f6-46f4-4a79-99f9-afd63890c6d0/data/beta/beta_wgs_98622.mt'
# The gnomAD v3.1.2 data set contains 76,156 whole genomes (and no exomes), all mapped to the GRCh38 reference sequence.
# See also https://gnomad.broadinstitute.org/downloads
GNOMAD_TAB = 'gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht'
# Matrix table was created from UKB 200k exome release VCFs.
# Note: The UKB matrix table was created via notebook 'aou_workbench_pooled_analyses/matrix_table_creation/create_matrix_tables.ipynb'
# and then repartitioned via notebook 'aou_workbench_pooled_analyses/matrix_table_creation/redo_partitions'.
UKB_MT = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/ukb/exomes/full_dataset_fewer_partitions.mt'
# Capture regions used for AoU WGS data.
WGS_CAPTURE_REGIONS = 'gs://gcp-public-data--broad-references/hg38/v0/wgs_calling_regions.hg38.noCentromeres.noTelomeres.interval_list'
# These matrix tables were created via notebook `aou_workbench_pooled_analyses/03_merge_variants.ipynb`. 
MERGED_MT = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220209/merged-aou3-ukb-filtered-chr1-chr22.mt'
AOU_ONLY_TAB = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220209/aou3-without_ukb-filtered-chr1-chr22.tab'
UKB_ONLY_TAB = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/pooled/geno/20220505/ukb-without_aou3-filtered-chr1-chr22.tab'

# Use autosomes only.
INTERVALS_TO_EXAMINE = ['chr1-chr22']
INTERVALS_TO_EXAMINE_NAME = '_'.join(INTERVALS_TO_EXAMINE).replace(':', 'range')

# Some cells are useful to run when testing over a small region of the genome, but unnecessary
# when running over all of the data.
SKIP_SOME_CELLS = True

#---[ Outputs ]---
# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
TIMESTAMP = time.strftime('%Y%m%d_%H%M%S')
WORK_DIR = !pwd

RESULTS_DIR = f'{os.getenv("WORKSPACE_BUCKET")}/data/pooled/examine-geno/{DATESTAMP}/'
MERGED_ALLELE_FREQUENCY = f'merged_allele_freq-{INTERVALS_TO_EXAMINE_NAME}.tsv'
AOU_ONLY_ALLELE_FREQUENCY = f'aou_only_allele_freq-{INTERVALS_TO_EXAMINE_NAME}.tsv'
UKB_ONLY_ALLELE_FREQUENCY = f'ukb_only_allele_freq-{INTERVALS_TO_EXAMINE_NAME}.tsv'
HAIL_LOG = f'{WORK_DIR[0]}/hail-examine-merged-variants-{TIMESTAMP}.log'
HAIL_LOG_DIR_FOR_PROVENANCE = f'{os.getenv("WORKSPACE_BUCKET")}/hail-logs/{DATESTAMP}/'

## Check access

In [ ]:
!gsutil ls {MERGED_MT}
#!gsutil du -sh {MERGED_MT}

In [ ]:
!gsutil ls {AOU_ONLY_TAB}
#!gsutil du -sh {AOU_ONLY_TAB}

In [ ]:
!gsutil ls {UKB_ONLY_TAB}
#!gsutil du -sh {UKB_ONLY_TAB}

## Start Hail 

In [ ]:
# See also https://towardsdatascience.com/fetch-failed-exception-in-apache-spark-decrypting-the-most-common-causes-b8dff21075c
# See https://spark.apache.org/docs/2.4.7/configuration.html

EXTRA_SPARK_CONFIG = {
    # If set to "true", performs speculative execution of tasks. This means if one or more tasks are running
    # slowly in a stage, they will be re-launched.
    'spark.speculation': 'true', # Default is false.
    
    # Fraction of tasks which must be complete before speculation is enabled for a particular stage.
    'spark.speculation.quantile': '0.95', # Default is 0.75

    # Default timeout for all network interactions. This config will be used in place of 
    # spark.core.connection.ack.wait.timeout, spark.storage.blockManagerSlaveTimeoutMs, 
    # spark.shuffle.io.connectionTimeout, spark.rpc.askTimeout or spark.rpc.lookupTimeout if they are not configured.
    'spark.network.timeout': '180s', # Default is 120s
        
    # (Netty only) Fetches that fail due to IO-related exceptions are automatically retried if this is set to a
    # non-zero value. This retry logic helps stabilize large shuffles in the face of long GC pauses or transient
    # network connectivity issues.
    'spark.shuffle.io.maxRetries': '10',  # Default is 3
    
    # (Netty only) How long to wait between retries of fetches. The maximum delay caused by retrying is 15 seconds
    # by default, calculated as maxRetries * retryWait.
    'spark.shuffle.io.retryWait': '15s',  # Default is 5s
    
    # Number of failures of any particular task before giving up on the job. The total number of failures spread
    # across different tasks will not cause the job to fail; a particular task has to fail this number of attempts.
    # Should be greater than or equal to 1. Number of allowed retries = this value - 1.
    'spark.task.maxFailures': '10', # Default is 4.

    # Number of consecutive stage attempts allowed before a stage is aborted.
    'spark.stage.maxConsecutiveAttempts': '10' # Default is 4.
}

In [ ]:
hl.init(spark_conf=EXTRA_SPARK_CONFIG,
        min_block_size=50,
        default_reference='GRCh38',
        log=HAIL_LOG)

Check the configuration.

In [ ]:
sc = hl.spark_context()
config = sc._conf.getAll()
config.sort()
config

In [ ]:
start_all = datetime.now()
print(start_all)

# Read merged matrix table

In [ ]:
merged = hl.read_matrix_table(MERGED_MT)

In [ ]:
merged.describe()

In [ ]:
merged.count()

In [ ]:
merged.n_partitions()

In [ ]:
merged = hl.filter_intervals(
    merged,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
merged.aggregate_cols(hl.agg.counter(merged.cohort))

In [ ]:
hl.summarize_variants(merged)

# Read AoU-only table

In [ ]:
aou_only = hl.read_table(AOU_ONLY_TAB)

In [ ]:
aou_only.describe()

In [ ]:
aou_only.count()

In [ ]:
aou_only.n_partitions()

In [ ]:
aou_only = hl.filter_intervals(
    aou_only,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
hl.summarize_variants(aou_only)

# Read UKB-only table

In [ ]:
ukb_only = hl.read_table(UKB_ONLY_TAB)

In [ ]:
ukb_only.describe()

In [ ]:
ukb_only.count()

In [ ]:
ukb_only.n_partitions()

In [ ]:
ukb_only = hl.filter_intervals(
    ukb_only,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
hl.summarize_variants(ukb_only)

# Read the gnomAD variant annotation table

In [ ]:
gnomad = hl.read_table(GNOMAD_TAB)

In [ ]:
gnomad.describe()

In [ ]:
gnomad.count()

In [ ]:
gnomad.n_partitions()

In [ ]:
gnomad = hl.filter_intervals(
    gnomad,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
gnomad.show()

# Also read the source data

For comparison purposes.

In [ ]:
aou_wgs = hl.read_matrix_table(AOU_MT)

In [ ]:
aou_wgs.describe()

In [ ]:
aou_wgs = hl.filter_intervals(
    aou_wgs,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
ukb_exomes = hl.read_matrix_table(UKB_MT)

In [ ]:
# https://discuss.hail.is/t/improving-pipeline-performance/1344
new_partitions = ukb_exomes._calculate_new_partitions(2500)
ukb_exomes = hl.read_matrix_table(UKB_MT, _intervals=new_partitions)

In [ ]:
ukb_exomes.describe()

In [ ]:
ukb_exomes = hl.filter_intervals(
    ukb_exomes,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

# Extract allele frequency and count data

Extract this information to CSV for plotting in R.

## Annotate with gnomAD

In [ ]:
# Confirm that the gnomad data is biallelic.
if not SKIP_SOME_CELLS:
    gnomad.group_by(num_alleles = hl.len(gnomad.alleles)).aggregate(n = hl.agg.count()).show()

In [ ]:
aou_only.count()

In [ ]:
# Check that we have reasonable overlap between gnomAD and AoU only.
if not SKIP_SOME_CELLS:
    gnomad.semi_join(aou_only).count()

In [ ]:
aou_only = aou_only.annotate(gnomad_popmax = gnomad[aou_only.key].popmax)

In [ ]:
aou_only.describe()

In [ ]:
aou_only.show()

In [ ]:
ukb_only = ukb_only.annotate(gnomad_popmax = gnomad[ukb_only.key].popmax)

In [ ]:
merged = merged.annotate_rows(gnomad_popmax = gnomad[merged.row_key].popmax)

## Write allele allele information to CSV

### Extract AoU-only allele information to a CSV

In [ ]:
aou_only_allele_freq = aou_only.select(
    AC = aou_only.aou_info.AC[aou_only.aou_a_index - 1],
    AF = aou_only.aou_info.AF[aou_only.aou_a_index - 1],
    AN = aou_only.aou_info.AN,
    gnomad_popmax_AC = aou_only.gnomad_popmax.AC,
    gnomad_popmax_AF = aou_only.gnomad_popmax.AF,
    gnomad_popmax_AN = aou_only.gnomad_popmax.AN,
    gnomad_popmax_pop = aou_only.gnomad_popmax.pop,
)

In [ ]:
aou_only_allele_freq.describe()

In [ ]:
aou_only_allele_freq.export(os.path.join(RESULTS_DIR, AOU_ONLY_ALLELE_FREQUENCY))

### Extract UKB-only allele information to a CSV

In [ ]:
ukb_only_allele_freq = ukb_only.select(
    AC = ukb_only.gt_stats.AC[1],
    AF = ukb_only.gt_stats.AF[1],
    AN = ukb_only.gt_stats.AN,
    gnomad_popmax_AC = ukb_only.gnomad_popmax.AC,
    gnomad_popmax_AF = ukb_only.gnomad_popmax.AF,
    gnomad_popmax_AN = ukb_only.gnomad_popmax.AN,
    gnomad_popmax_pop = ukb_only.gnomad_popmax.pop,
)

In [ ]:
ukb_only_allele_freq.describe()

In [ ]:
ukb_only_allele_freq.export(os.path.join(RESULTS_DIR, UKB_ONLY_ALLELE_FREQUENCY))

### Extract merged allele information to a CSV

In [ ]:
merged_rows = merged.rows()
merged_allele_freq = merged_rows.select(
    AC = merged_rows.gt_stats.AC[1],
    AF = merged_rows.gt_stats.AF[1],
    AN = merged_rows.gt_stats.AN,
    gnomad_popmax_AC = merged_rows.gnomad_popmax.AC,
    gnomad_popmax_AF = merged_rows.gnomad_popmax.AF,
    gnomad_popmax_AN = merged_rows.gnomad_popmax.AN,
    gnomad_popmax_pop = merged_rows.gnomad_popmax.pop,
)

In [ ]:
merged_allele_freq.describe()

In [ ]:
merged_allele_freq.export(os.path.join(RESULTS_DIR, MERGED_ALLELE_FREQUENCY))

In [ ]:
!gsutil ls {RESULTS_DIR}

# Examine the data

## Are the WGS capture regions yielding any unmerged UKB variants?

**Answer**: none!

In [ ]:
aou_wgs_capture_regions = hl.import_locus_intervals(WGS_CAPTURE_REGIONS)

In [ ]:
aou_wgs_capture_regions.describe()

In [ ]:
aou_wgs_capture_regions.show(5)

In [ ]:
aou_wgs_capture_regions.aggregate(hl.agg.counter(aou_wgs_capture_regions.interval.start.contig))

In [ ]:
ukb_only_outside_wgs_capture_region = ukb_only.filter(
    hl.is_defined(aou_wgs_capture_regions[ukb_only.locus]), keep = False)

In [ ]:
ukb_only_outside_wgs_capture_region.count()

## How many merged variants were flagged by AoU filters?

In [ ]:
merged_rows = merged.rows()
merged_rows.group_by(merged_rows.aou_filters).aggregate(n = hl.agg.count()).show()

## How many merged variants were flagged by UKB filters?

In [ ]:
ukb_exomes_rows = ukb_exomes.rows()

In [ ]:
ukb_exomes_rows_has_filter_flag = ukb_exomes_rows.filter(hl.is_defined(ukb_exomes_rows.filters), keep = True)

In [ ]:
# Split the multi-allelic sites into biallelic sites.
ukb_exomes_rows_has_filter_flag = hl.split_multi_hts(ukb_exomes_rows_has_filter_flag_multi,
                                                     keep_star=False,
                                                     left_aligned=False,
                                                     vep_root='vep',
                                                     permit_shuffle=False)

ukb_exomes_rows_has_filter_flag.describe()

In [ ]:
merged_rows = merged_rows.annotate(ukb_filter = ukb_exomes_rows_has_filter_flag[merged_rows.key].filters)
merged_rows.describe()

In [ ]:
merged_rows.group_by(merged_rows.ukb_filter).aggregate(n = hl.agg.count()).show()

## Are the unmerged variants mostly rare?


**Answer**: yes for UKB, somewhat for AoU

### Allele frequencies at different quantiles.

In [ ]:
aou_only.aggregate(hl.agg.approx_quantiles(
    aou_only.aou_info.AF[aou_only.aou_a_index - 1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.ukb_info.AF[ukb_only.ukb_a_index - 1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.gt_stats.AF[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

### Allele count and number

In [ ]:
aou_only.aggregate(hl.agg.approx_quantiles(
    aou_only.aou_info.AC[aou_only.aou_a_index - 1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
aou_only.aggregate(hl.agg.approx_quantiles(
    aou_only.aou_info.AN,
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

<div class="alert alert-block alert-info">
<b>Note:</b> The original UKB VCFs list info fields for AC and AN but their values are not populated.
</div>

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.ukb_info.AC[ukb_only.ukb_a_index - 1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.gt_stats.AC[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.ukb_info.AN,
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only.aggregate(hl.agg.approx_quantiles(
    ukb_only.gt_stats.AN,
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

What about for the merged data?

In [ ]:
merged.aggregate_rows(hl.agg.approx_quantiles(
    merged.gt_stats.AC[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
merged.aggregate_rows(hl.agg.approx_quantiles(
    merged.gt_stats.AF[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
merged.aggregate_rows(hl.agg.approx_quantiles(
    merged.gt_stats.AF[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

## How many unmerged variants are common?

**Answer**: a few for each

### Common AoU only

In [ ]:
aou_only_common = aou_only.filter(
    aou_only.aou_info.AF[aou_only.aou_a_index - 1] > 0.01,
    keep=True)
aou_only_common = aou_only_common.filter(
    aou_only_common.aou_info.AF[aou_only_common.aou_a_index - 1] < 0.99,
    keep=True)

In [ ]:
aou_only_common.count()

In [ ]:
hl.summarize_variants(aou_only_common)

In [ ]:
aou_only_common.group_by(aou_only_common.aou_filters).aggregate(n = hl.agg.count()).show()

In [ ]:
aou_only_common.filter(aou_only_common.aou_filters.contains('NO_HQ_GENOTYPES')).show()

In [ ]:
aou_only_common.aggregate(hl.agg.approx_quantiles(
    aou_only_common.aou_info.AC[aou_only_common.aou_a_index - 1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
aou_only_common.aggregate(hl.agg.approx_quantiles(
    aou_only_common.aou_info.AN,
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

### Common UKB only

In [ ]:
ukb_only_common = ukb_only.filter(
    ukb_only.ukb_info.AF[ukb_only.ukb_a_index - 1] > 0.01,
    keep=True)
ukb_only_common = ukb_only_common.filter(
    ukb_only_common.ukb_info.AF[ukb_only_common.ukb_a_index - 1] < 0.99,
    keep=True)

In [ ]:
ukb_only_common_count = ukb_only_common.count()

ukb_only_common_count

In [ ]:
if ukb_only_common_count > 0:
    hl.summarize_variants(ukb_only_common)

In [ ]:
ukb_only_common.aggregate(hl.agg.approx_quantiles(
    ukb_only_common.gt_stats.AC[1],
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

In [ ]:
ukb_only_common.aggregate(hl.agg.approx_quantiles(
    ukb_only_common.gt_stats.AN,
    [0, 0.25, 0.5, 0.75, .90, .99, .999, 1]
))

### Common merged

In [ ]:
merged_common = merged.filter_rows(merged.gt_stats.AF[1] > 0.01, keep=True)
merged_common = merged_common.filter_rows(merged_common.gt_stats.AF[1] < 0.99, keep=True)

In [ ]:
merged_common.count()

In [ ]:
hl.summarize_variants(merged_common)

## Define the variant missingness threshold

Many of the common variants are actually quite rare because AN is small. Let's only examine those that have less missingness than our QC threshold. 

In [ ]:
missingness_AN_threshold = math.ceil(merged.count_cols() * 2 * .1) # 10%
missingness_AN_threshold

## For the truly common unmerged AoU variants, what do the overlapping UKB variants look like?

**Answer**: see details below

### Show common unmerged from AoU

<div class="alert alert-block alert-success">
Sorted by AF, decending.</div>


In [ ]:
aou_only_common.order_by(
    hl.desc(aou_only_common.aou_info.AF[aou_only_common.aou_a_index - 1])).show(20)

In [ ]:
aou_only_truly_common = aou_only_common.filter(aou_only_common.aou_info.AN > missingness_AN_threshold, keep=True)
aou_only_truly_common = aou_only_truly_common.filter(hl.is_missing(aou_only_truly_common.aou_filters), keep=True)

In [ ]:
aou_only_truly_common.order_by(hl.desc(aou_only_truly_common.aou_info.AF[aou_only_truly_common.aou_a_index - 1])).show(20)

In [ ]:
aou_only_common_top20_loci = aou_only_truly_common.order_by(
        hl.desc(aou_only_truly_common.aou_info.AF[aou_only_truly_common.aou_a_index - 1])).locus.take(20)

<div class="alert alert-block alert-success">
Of those top 20 by AF, now sorted by position, ascending.</div>

In [ ]:
aou_only_common_top20 = aou_only_common.filter(
    hl.literal(aou_only_common_top20_loci).contains(aou_only_common.locus))
aou_only_common_top20 = aou_only_common_top20.select(
    aou_only_common_top20.aou_was_split,
    aou_only_common_top20.aou_filters,
    AF = aou_only_common_top20.aou_info.AF[aou_only_common_top20.aou_a_index - 1]
    )

aou_only_common_top20.order_by(aou_only_common_top20.locus).show(50)

<div class="alert alert-block alert-success">
    Show UKB <b>source data</b> locus matches.
    </div>

In [ ]:
ukb_locus_matches = ukb_exomes.filter_rows(
    hl.literal(aou_only_common_top20_loci).contains(ukb_exomes.locus)).rows()
ukb_locus_matches = ukb_locus_matches.select(
    ukb_locus_matches.filters,
    ukb_locus_matches.info.AF
)

ukb_locus_matches.order_by(ukb_locus_matches.locus).show(50)

<div class="alert alert-block alert-success">
    Show UKB<b>-only</b> locus matches.
    </div>

In [ ]:
ukb_only_locus_matches = ukb_only.filter(
    hl.literal(aou_only_common_top20_loci).contains(ukb_only.locus))
ukb_only_locus_matches = ukb_only_locus_matches.select(
    ukb_only_locus_matches.ukb_was_split,
    AF = ukb_only_locus_matches.ukb_info.AF[ukb_only_locus_matches.ukb_a_index - 1]
)

ukb_only_locus_matches.order_by(ukb_only_locus_matches.locus).show(50)

<div class="alert alert-block alert-success">
    Show AoU <b>source data</b> locus matches.
    </div>

In [ ]:
aou_locus_matches = aou_wgs.filter_rows(
    hl.literal(aou_only_common_top20_loci).contains(aou_wgs.locus)).rows()
aou_locus_matches = aou_locus_matches.select(aou_locus_matches.info.AF)

aou_locus_matches.order_by(aou_locus_matches.locus).show(50)

### Examine a common AoU SNP

In [ ]:
[(x, aou_only_common_top20_loci[x]) for x in range(0, len(aou_only_common_top20_loci))]

In [ ]:
one_aou_variant = [aou_only_common_top20_loci[4]]

one_aou_variant

In [ ]:
ukb_exomes.filter_rows(
    hl.literal(one_aou_variant).contains(ukb_exomes.locus)).show(20)

In [ ]:
ukb_nearby_common_aou = hl.filter_intervals(
     ukb_exomes,
     [hl.interval(hl.locus(x.contig, x.position - 20), hl.locus(x.contig, x.position + 20),
                  includes_start=True, includes_end=True) for x in one_aou_variant])

In [ ]:
ukb_nearby_common_aou.show(20)

## For the truly common unmerged UKB variants, what do the overlapping AoU variants look like?

**Answer**: see details below

### Show common unmerged from UKB

<div class="alert alert-block alert-success">
Sorted by AF, decending.</div>


In [ ]:
if ukb_only_common_count > 0:
    ukb_only_common.order_by(
        hl.desc(ukb_only_common.ukb_info.AF[ukb_only_common.ukb_a_index - 1])).show(20)

In [ ]:
if ukb_only_common_count > 0:
    ukb_only_truly_common = ukb_only_common.filter(ukb_only_common.ukb_info.AN > missingness_AN_threshold, keep=True)
    ukb_only_truly_common_count = ukb_only_truly_common.count()
    ukb_only_truly_common.order_by(hl.desc(ukb_only_truly_common.ukb_info.AF[ukb_only_truly_common.ukb_a_index - 1])).show(20)

In [ ]:
if ukb_only_truly_common_count > 0:
    ukb_only_common_top20_loci = ukb_only_truly_common.order_by(
        hl.desc(ukb_only_truly_common.ukb_info.AF[ukb_only_truly_common.ukb_a_index - 1])).locus.take(20)

<div class="alert alert-block alert-success">
Of those top 20 by AF, now sorted by position, ascending.</div>

In [ ]:
if ukb_only_truly_common_count > 0:
    ukb_only_common_top20 = ukb_only_common.filter(
        hl.literal(ukb_only_common_top20_loci).contains(ukb_only_common.locus))
    ukb_only_common_top20 = ukb_only_common_top20.select(
        ukb_only_common_top20.ukb_was_split,
        ukb_only_common_top20.ukb_filters,
        AF = ukb_only_common_top20.ukb_info.AF[ukb_only_common_top20.ukb_a_index - 1]
    )

    ukb_only_common_top20.order_by(ukb_only_common_top20.locus).show(50)

<div class="alert alert-block alert-success">
    Show AoU <b>source data</b> locus matches.
    </div>

In [ ]:
if ukb_only_truly_common_count > 0:
    aou_locus_matches = aou_wgs.filter_rows(
        hl.literal(ukb_only_common_top20_loci).contains(aou_wgs.locus)).rows()
    aou_locus_matches = aou_locus_matches.select(
        aou_locus_matches.filters,
        aou_locus_matches.info.AF
    )

    aou_locus_matches.order_by(aou_locus_matches.locus).show(50)

<div class="alert alert-block alert-success">
    Show AoU<b>-only</b> locus matches.
    </div>

In [ ]:
if ukb_only_truly_common_count > 0:
    aou_only_locus_matches = aou_only.filter(
        hl.literal(ukb_only_common_top20_loci).contains(aou_only.locus))
    aou_only_locus_matches = aou_only_locus_matches.select(
        aou_only_locus_matches.aou_was_split,
        AF = aou_only_locus_matches.aou_info.AF[aou_only_locus_matches.aou_a_index - 1]
    )

    aou_only_locus_matches.order_by(aou_only_locus_matches.locus).show(50)

<div class="alert alert-block alert-success">
    Show UKB <b>source data</b> locus matches.
    </div>

In [ ]:
if ukb_only_truly_common_count > 0:
    ukb_locus_matches = ukb_exomes.filter_rows(
        hl.literal(ukb_only_common_top20_loci).contains(ukb_exomes.locus)).rows()
    ukb_locus_matches = ukb_locus_matches.select(ukb_locus_matches.info.AF)

    ukb_locus_matches.order_by(ukb_locus_matches.locus).show(50)

### Examine a common UKB SNP

In [ ]:
if ukb_only_truly_common_count > 0:
    [(x, ukb_only_common_top20_loci[x]) for x in range(0, len(ukb_only_common_top20_loci))]

In [ ]:
if ukb_only_truly_common_count > 0:
    one_ukb_variant = [ukb_only_common_top20_loci[0]]

    one_ukb_variant

In [ ]:
if ukb_only_truly_common_count > 0:
    aou_wgs.filter_rows(
        hl.literal(one_ukb_variant).contains(aou_wgs.locus)).show(20)

In [ ]:
if ukb_only_truly_common_count > 0:
    aou_nearby_common_ukb = hl.filter_intervals(
         aou_wgs,
         [hl.interval(hl.locus(x.contig, x.position - 20), hl.locus(x.contig, x.position + 20),
                      includes_start=True, includes_end=True) for x in one_ukb_variant])

In [ ]:
if ukb_only_truly_common_count > 0:
    aou_nearby_common_ukb.show(20)

# Provenance

In [ ]:
end_all = datetime.now()
print(end_all)
print(end_all - start_all)

In [ ]:
# Copy the Hail log to the workspace bucket so that we can retain it.
!gzip --keep {HAIL_LOG}
!gsutil cp -v {HAIL_LOG}.gz {HAIL_LOG_DIR_FOR_PROVENANCE}

In [ ]:
print(datetime.now())

In [ ]:
!pip3 freeze